In [14]:
import numpy as np
import json
from pathlib import Path
import scipy
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sentence_transformers import SentenceTransformer
from xgboost import XGBClassifier

ImportError: cannot import name 'get_test_and_labels' from 'funkisar' (c:\Users\maxrl\OneDrive\Dokument\EP\INF554\challange\download\funkisar.py)

In [2]:
def flatten(list_of_list):
    return [item for sublist in list_of_list for item in sublist]

path_to_training = Path("training")
path_to_test = Path("test")

validation_set=['ES2006']
training_set = ['ES2002', 'ES2005', 'ES2006', 'ES2007', 'ES2008', 'ES2009', 'ES2010', 'ES2012', 'ES2013', 'ES2015', 'ES2016', 'IS1000', 'IS1001', 'IS1002', 'IS1003', 'IS1004', 'IS1005', 'IS1006', 'IS1007', 'TS3005', 'TS3008', 'TS3009', 'TS3010', 'TS3011', 'TS3012']
for element in validation_set:
    training_set.remove(element)
training_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in training_set])
training_set.remove('IS1002a')
training_set.remove('IS1005d')
training_set.remove('TS3012c')


validation_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in validation_set])



X_validation, y_validation = get_test_and_labels(validation_set)
X_training, y_training = get_test_and_labels(training_set)



test_set = ['ES2003', 'ES2004', 'ES2011', 'ES2014', 'IS1008', 'IS1009', 'TS3003', 'TS3004', 'TS3006', 'TS3007']
test_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in test_set])

test_labels = {}
for transcription_id in test_set:
    with open(path_to_test / f"{transcription_id}.json", "r") as file:
        transcription = json.load(file)
    
    test_labels[transcription_id] = [1] * len(transcription)

with open("test_labels_naive_baseline.json", "w") as file:
    json.dump(test_labels, file, indent=4)



In [4]:
bert = SentenceTransformer('all-MiniLM-L6-v2')
print(bert.device)

X_validation = bert.encode(X_validation, show_progress_bar=True)
X_training = bert.encode(X_training, show_progress_bar=True)



Batches:   0%|          | 0/98 [00:00<?, ?it/s]

Batches:   0%|          | 0/2172 [00:00<?, ?it/s]

In [5]:


speaker_validation=get_speaker(validation_set)
speaker_training=get_speaker(training_set)


X_training=np.c_[X_training, speaker_training]
X_validation=np.c_[X_validation, speaker_validation]



In [6]:

len_validation=get_length(validation_set)
len_training=get_length(training_set)


X_training=np.c_[X_training, len_training]
X_validation=np.c_[X_validation, len_validation]

In [7]:
bst = XGBClassifier()
# fit model
bst.fit(X_training, y_training)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

9
0
8
1
4
5
2
3
7
6


1
0
tensor([1, 2, 3])
Device:  cpu
tensor([4, 5, 6], device='cuda:0')
Device:  cuda:0
